In [ ]:
import pandas as pd
import pdfplumber

# 1. Read and group by ISIN in Excel
excel_file = 'your_isin_file.xlsx'
df_excel = pd.read_excel(excel_file)

# Group by ISIN and sum the amount
df_grouped = df_excel.groupby('ISIN')['Amount'].sum().reset_index()

# 2. Extract amount from PDF (assuming a table structure)
pdf_file = 'your_client_pdf.pdf'
with pdfplumber.open(pdf_file) as pdf:
    for page in pdf.pages:
        table = page.extract_table()
        # Assuming ISIN and amount are in the first and second columns respectively
        pdf_data = pd.DataFrame(table[1:], columns=table[0])  # convert to DataFrame for easier manipulation
        # Find the row where ISIN matches
        pdf_isin_row = pdf_data[pdf_data['Fund ISIN'] == 'Your ISIN']

# Get the relevant amount from PDF row
pdf_isin_amount = pdf_isin_row['Amount'].astype(float).sum()

# 3. Create comparison DataFrame
comparison_df = pd.DataFrame({
    'ISIN': df_grouped['ISIN'],
    'Excel Amount': df_grouped['Amount'],
    'PDF Amount': pdf_isin_amount
})

# Calculate the difference and percentage difference
comparison_df['Difference'] = comparison_df['Excel Amount'] - comparison_df['PDF Amount']
comparison_df['% Difference'] = (comparison_df['Difference'] / comparison_df['Excel Amount']) * 100

print(comparison_df)
